In [1]:
import os
import urllib
import shutil
import azureml
import pandas as pd

from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Experiment, Workspace, Run, Dataset

In [2]:
import argparse
import os
import pandas as pd
import numpy as np
import pickle
import json

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

import azureml.core
from azureml.core import Run
from azureml.core.model import Model
from azureml.core import Workspace, Dataset
from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

In [3]:
import os
from azureml.core.authentication import ServicePrincipalAuthentication
 
svc_pr_password = os.environ.get("AZUREML_PASSWORD")
 
svc_pr = ServicePrincipalAuthentication(
    tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47",
    service_principal_id="8a3ddafe-6dd6-48af-867e-d745232a1833",
    service_principal_password="1fY58u0dpP1Yg-i.A~rUp_iz04RxWUFSwv")
 
ws = Workspace(
    subscription_id="c46a9435-c957-4e6c-a0f4-b9a597984773",
    resource_group="mlops",
    workspace_name="mlopsdev",
    auth=svc_pr
    )
 
print("Found workspace {} at location {}".format(ws.name, ws.location))

Found workspace mlopsdev at location centralus


In [4]:
project_folder = './diabetes-project'
os.makedirs(project_folder, exist_ok=True)

experiment = Experiment(workspace=ws, name='diabetes-model')

In [5]:
output_folder = './outputs'
os.makedirs(output_folder, exist_ok=True)

In [6]:
result_folder = './results'
os.makedirs(result_folder, exist_ok=True)

In [7]:
df = pd.read_csv('https://mlopssa.blob.core.windows.net/chd-dataset/framingham.csv')

In [8]:
df.columns

Index(['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD'],
      dtype='object')

In [9]:
# create a boolean array of smokers
smoke = (df['currentSmoker']==1)
# Apply mean to NaNs in cigsPerDay but using a set of smokers only
df.loc[smoke,'cigsPerDay'] = df.loc[smoke,'cigsPerDay'].fillna(df.loc[smoke,'cigsPerDay'].mean())

# Fill out missing values
df['BPMeds'].fillna(0, inplace = True)
df['glucose'].fillna(df.glucose.mean(), inplace = True)
df['totChol'].fillna(df.totChol.mean(), inplace = True)
df['education'].fillna(1, inplace = True)
df['BMI'].fillna(df.BMI.mean(), inplace = True)
df['heartRate'].fillna(df.heartRate.mean(), inplace = True)

# Features and label
features = df.iloc[:,:-1]
result = df.iloc[:,-1] # the last column is what we are about to forecast

In [10]:
# Train & Test split
X_train, X_test, y_train, y_test = train_test_split(features, result, test_size = 0.2, random_state = 14)

In [11]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [12]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [13]:
# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.12
sfm = SelectFromModel(clf, threshold=0.12)

# Train the selector
sfm.fit(X_train, y_train)

# Features selected
featureNames = list(features.columns.values) # creating a list with features' names
print("Feature names:")
for featureNameListindex in sfm.get_support(indices=True):
    print(featureNames[featureNameListindex])

# Feature importance
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# With only imporant features. Can check X_important_train.shape[1]
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)
#Y_important_test = sfm.transform(y_test)

rfc = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
rfc.fit(X_important_train, y_train)

Feature names:
age
prevalentHyp
sysBP
glucose


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [14]:
preds = rfc.predict(X_important_test)

In [18]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, preds))

Accuracy: 0.8290094339622641


In [19]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "touringcluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D14_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [20]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

# Create a new runconfig object
run_amlcompute = RunConfiguration()

# Use the cpu_cluster you created above. 
run_amlcompute.target = cpu_cluster

# Enable Docker
run_amlcompute.environment.docker.enabled = True

# Set Docker base image to the default CPU-based image
run_amlcompute.environment.docker.base_image = DEFAULT_CPU_IMAGE

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
run_amlcompute.environment.python.user_managed_dependencies = False

# Specify CondaDependencies obj, add necessary packages
run_amlcompute.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [21]:
%%writefile $project_folder/train.py

import joblib
import os
import urllib
import shutil
import azureml
import argparse
import pandas as pd
import numpy as np
import pickle
import json
from sklearn import metrics

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from sklearn.ensemble import RandomForestClassifier


from azureml.core import Workspace, Dataset

from azureml.core.authentication import ServicePrincipalAuthentication
 
svc_pr_password = "1fY58u0dpP1Yg-i.A~rUp_iz04RxWUFSwv"
 
svc_pr = ServicePrincipalAuthentication(
    tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47",
    service_principal_id="8a3ddafe-6dd6-48af-867e-d745232a1833",
    service_principal_password="1fY58u0dpP1Yg-i.A~rUp_iz04RxWUFSwv")
 
ws = Workspace(
    subscription_id="c46a9435-c957-4e6c-a0f4-b9a597984773",
    resource_group="mlops",
    workspace_name="mlopsdev",
    auth=svc_pr
    )

#dataset = Dataset.get_by_name(ws, name='touringdataset')
#dataset.to_pandas_dataframe()
data_complete = df = pd.read_csv('https://mlopssa.blob.core.windows.net/chd-dataset/framingham.csv')

# Get the experiment run context
run = Run.get_context()

# create a boolean array of smokers
smoke = (df['currentSmoker']==1)
# Apply mean to NaNs in cigsPerDay but using a set of smokers only
df.loc[smoke,'cigsPerDay'] = df.loc[smoke,'cigsPerDay'].fillna(df.loc[smoke,'cigsPerDay'].mean())

# Fill out missing values
df['BPMeds'].fillna(0, inplace = True)
df['glucose'].fillna(df.glucose.mean(), inplace = True)
df['totChol'].fillna(df.totChol.mean(), inplace = True)
df['education'].fillna(1, inplace = True)
df['BMI'].fillna(df.BMI.mean(), inplace = True)
df['heartRate'].fillna(df.heartRate.mean(), inplace = True)

# Features and label
features = df.iloc[:,:-1]
result = df.iloc[:,-1] # the last column is what we are about to forecast

# Train & Test split
X_train, X_test, y_train, y_test = train_test_split(features, result, test_size = 0.2, random_state = 14)

# RandomForest classifier
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)

# Create a selector object that will use the random forest classifier to identify
# features that have an importance of more than 0.12
sfm = SelectFromModel(clf, threshold=0.12)

# Train the selector
sfm.fit(X_train, y_train)

# Features selected
featureNames = list(features.columns.values) # creating a list with features' names
print("Feature names:")
for featureNameListindex in sfm.get_support(indices=True):
    print(featureNames[featureNameListindex])

# Feature importance
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# With only imporant features. Can check X_important_train.shape[1]
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)

rfc = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
rfc.fit(X_important_train, y_train)

preds = rfc.predict(X_important_test)

run.log("Accuracy:",metrics.accuracy_score(y_test, preds))

#joblib.dump(rfc, "/outputs/model.joblib")
os.makedirs('./outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=rfc, filename='./outputs/sklearn_diabetes_model.pkl')

Overwriting ./diabetes-project/train.py


In [22]:
from azureml.train.sklearn import SKLearn

# script_params = {
#     '--kernel': 'linear',
#     '--penalty': 1.0,
# }

estimator = SKLearn(source_directory=project_folder, 
#                     script_params=script_params,
                    compute_target=cpu_cluster,
                    entry_script='train.py',
                    pip_packages=['joblib']
                   )

WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['joblib']. We cannot guarantee image build will succeed.


In [23]:
run = experiment.submit(estimator)
run.wait_for_completion(show_output=True)

RunId: diabetes-model_1593963743_6a58ce3e
Web View: https://ml.azure.com/experiments/diabetes-model/runs/diabetes-model_1593963743_6a58ce3e?wsid=/subscriptions/c46a9435-c957-4e6c-a0f4-b9a597984773/resourcegroups/mlops/workspaces/mlopsdev

Streaming azureml-logs/55_azureml-execution-tvmps_a9d7b207ea4c2057530f01bb2bdc29f33eb044b41f5f7c5e3ad45d5f73f8f56a_d.txt

2020-07-05T15:45:46Z Starting output-watcher...
2020-07-05T15:45:46Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
181260fc7be4734cfa15a6febd1d9018a6704c8696d020d9440ae71998ff4c80

Streaming azureml-logs/65_job_prep-tvmps_a9d7b207ea4c2057530f01bb2bdc29f33eb044b41f5f7c5e3ad45d5f73f8f56a_d.txt

Entering job preparation. Current time:2020-07-05T15:46:24.755588
Starting job preparation. Current time:2020-07-05T15:46:26.916683
Extracting the control code.
fetching and extracting the control code on master node.

Streaming azureml-logs/70_driver_log.txt

Entering context manager injector. Current time:2020-07-05T15:46:31.

{'runId': 'diabetes-model_1593963743_6a58ce3e',
 'target': 'touringcluster',
 'status': 'Completed',
 'startTimeUtc': '2020-07-05T15:45:32.508132Z',
 'endTimeUtc': '2020-07-05T15:47:33.841745Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'ac58f8ee-dac5-48c1-8ca5-41e9222e9a34',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'touringcluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment diabetes-model Environment',
   'version': 'Autosave_2020-07-05T13:16:12Z_d1a5471c',
   'python': {'interpreterPath': 'python',
    'userManagedDependencies': False,
    'con

In [24]:
print(run.get_metrics())

{'Accuracy:': 0.8290094339622641}


In [25]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [26]:
# register model
model = run.register_model(model_name='sklearn_diabetes',
                           model_path='outputs/sklearn_diabetes_model.pkl',
                           tags=run.get_metrics())
print(model.name, model.id, model.version, sep='\t')

sklearn_diabetes	sklearn_diabetes:2	2
